In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pm4py
import scipy
import stormpy
from copy import deepcopy
import numpy as np

In [ ]:
from simulation.markov_models import log_parser
from simulation.markov_chain import apply as mc_apply
from simulation.markov_chain_vis import view_markov_chain, view_resource_markov_chain, view_non_resource_markov_chain
import simulation.util as sim_util

# Event log loading and processing

In [ ]:

import numpy as np
from scipy.stats import expon
from simulation.unfold_events import rename_repeating_events

# Simulation parameters
np.random.seed(42)

event_log = pm4py.read_xes('BPI_Challenge_2013_incidents.xes.gz')
event_log = event_log.sort_values(['case:concept:name','time:timestamp'])
number_of_traces = event_log['case:concept:name'].nunique()
subset_el = event_log[['case:concept:name','concept:name','time:timestamp','org:resource','org:role']]
subset_el['org:role'] = subset_el['org:role'].fillna('nan_1').apply(lambda x: x.split('_')[0])
subset_el['org:role'] = subset_el['org:role'].replace({'C':'C1','D':'D1','E':'E1'})
df = subset_el

epsilon = 1
final_states = ['Completed']
if epsilon>1:
    df, final_states = rename_repeating_events(df, epsilon, final_states)
subset_el = df
# Time unit configuration: choose "seconds" or "hours"
time_unit = "hours"  # or "seconds"
time_factor = 1 if time_unit == "seconds" else 1 / 3600  # seconds to chosen unit

In [ ]:
subset_el['concept:name'].unique()

In [ ]:
from simulation.timings import Timings

timings = Timings()
resource_input_array = timings.create_resource_input_array_from_log(subset_el)
res_timings = timings.get_timings_per_resource(subset_el, resource_input_array,time_factor=time_factor)
times_dictionary = res_timings

In [ ]:
print(len(times_dictionary.keys()))
print(times_dictionary.keys())

In [ ]:
time_diffs = times_dictionary

In [ ]:
plot = False

# Exponential fit and statistical hypothesis tests

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import expon, sem, t, entropy, kstest
from scipy.special import rel_entr, kl_div
from scipy.stats import wasserstein_distance
from datetime import datetime, timedelta

def compute_freedman_diaconis_bins(data):
    iqr = np.subtract(*np.percentile(data, [75, 25]))
    bin_width = 2 * iqr / (n ** (1/3))
    return int(np.ceil((np.max(data) - np.min(data)) / bin_width)) if bin_width > 0 else 1

# ---- Plot and Compute Distances ----
if plot:
    fig, axes = plt.subplots(1, len(time_diffs), figsize=(25, 5), tight_layout=True, sharey=True)
    if len(time_diffs) == 1:
        axes = [axes]

metrics_all = {}

for i, (pair, deltas) in enumerate(time_diffs.items()):
    deltas = np.array(deltas)
    mean_time = deltas.mean()
    print(pair, mean_time)
    n = len(deltas)

    # Manual fit
    rate_manual = 1 / mean_time
    ci_half_width = t.ppf(0.975, df=n-1) * sem(deltas)
    lower_rate = 1 / (mean_time + ci_half_width)
    upper_rate = 1 / (mean_time - ci_half_width)

    # Scipy fit
    loc, scale = expon.fit(deltas, floc=0)
    rate_scipy = 1 / scale

    # Histogram
    num_bins = compute_freedman_diaconis_bins(deltas)
    counts, bin_edges = np.histogram(deltas, bins=num_bins, density=False)
    bin_widths = np.diff(bin_edges)
    total = np.sum(counts)
    hist_probs = counts / total

    # Model probabilities over bins
    model_probs = expon.cdf(bin_edges[1:], scale=1 / rate_scipy) - expon.cdf(bin_edges[:-1], scale=1 / rate_scipy)
    eps = 1e-12
    # kldiv = np.sum(rel_entr(hist_probs, model_probs))
    test_kl_div = entropy(hist_probs, model_probs)
    test2_kl_div = np.sum(kl_div(hist_probs, model_probs))
    ks_test, ks_p_value = kstest(deltas, lambda deltas: expon.cdf(deltas,loc,scale))
    m = 0.5 * (hist_probs + model_probs)
    js_div = 0.5 * (entropy(hist_probs + eps, m + eps) + entropy(model_probs + eps, m + eps))
    tv_dist = 0.5 * np.sum(np.abs(hist_probs - model_probs))
    w_dist = wasserstein_distance(deltas, expon.rvs(scale=1 / rate_scipy, size=len(deltas), random_state=42))

    # Save metrics
    metrics_all[pair] = {
        "kl_divergence": test_kl_div,
        "js_divergence": js_div,
        "total_variation": tv_dist,
        "wasserstein_distance": w_dist,
        "ks_test": ks_test,
        "ks_p_value": ks_p_value,
        "n": n
    }

    # Plot histogram and fits
    if plot:
        ax = axes[i]
        sns.histplot(deltas, bins=num_bins, stat="density", ax=ax, color="skyblue", label="Empirical")
        x_vals = np.linspace(0, max(deltas) * 1.2, 200)
        ax.plot(x_vals, expon.pdf(x_vals, scale=1 / rate_scipy), linestyle="-.", color="purple",
                label=f"Fitted λ = {rate_scipy:.4f}")
        ax.fill_between(x_vals,
                        expon.pdf(x_vals, scale=1 / lower_rate),
                        expon.pdf(x_vals, scale=1 / upper_rate),
                        color="gray", alpha=0.3, label="95% CI band")

        ax.set_title(f"{pair[0]} → {pair[2]} → {pair[1]}")
        ax.set_xlabel(f"Time ({time_unit})")
        ax.set_ylabel("Density")
        ax.legend()

        # Text box with metrics
        textstr = 'Metrics:\n'
        textstr += '\n'.join([
            f"KL: {test_kl_div:.4f}",
            f"JS: {js_div:.4f}",
            f"TV: {tv_dist:.4f}",
            f"W: {w_dist:.4f}",
            f"KS: {ks_test:.4f} P: {ks_p_value:.4f}",
            f"N: {n}"
        ])
        ax.text(0.98, 0.55, textstr,
                transform=ax.transAxes,
                fontsize=9,
                verticalalignment='top',
                horizontalalignment='right',
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
if plot:
    plt.tight_layout()
    plt.show()

# If needed later:
# print(metrics_all)

In [ ]:
from scipy.stats import combine_pvalues

total_n = sum(m["n"] for m in metrics_all.values())

# Weighted averages for divergence metrics
kl_weighted = sum(m["kl_divergence"] * m["n"] for m in metrics_all.values()) / total_n
js_weighted = sum(m["js_divergence"] * m["n"] for m in metrics_all.values()) / total_n

# Simple averages for bounded or scale-sensitive distances
tv_average = np.mean([m["total_variation"] for m in metrics_all.values()])
w_average = np.mean([m["wasserstein_distance"] for m in metrics_all.values()])

ks_average = np.mean([m["ks_test"] for m in metrics_all.values()])
stat, ks_combined_p = combine_pvalues([m["ks_p_value"]+eps for m in metrics_all.values()])
# Final aggregate summary
aggregate_metrics = {
    "KL Divergence (weighted)": kl_weighted,
    "JS Divergence (weighted)": js_weighted,
    "Total Variation Distance (mean)": tv_average,
    "Wasserstein Distance (mean)": w_average,
    "KS Test (mean)": ks_average,
    "KS p-value (combined fisher)": ks_combined_p,
    "Total Samples": total_n
}

# Print aggregated metrics
print("\n--- Aggregated Metrics ---")
for key, val in aggregate_metrics.items():
    print(f"{key}: {val:.4f}")

# ---- Display in a New Figure ----
fig_summary, ax_summary = plt.subplots(figsize=(5, 4))
ax_summary.axis("off")

summary_text = '\n'.join([
    f"{k}: {v:.4f}" if isinstance(v, float) else f"{k}: {v}"
    for k, v in aggregate_metrics.items()
])

ax_summary.text(0.5, 0.5, summary_text,
                fontsize=11,
                ha="center",
                va="center",
                bbox=dict(boxstyle="round", facecolor="white", alpha=0.9))

ax_summary.set_title("Aggregated Fit Metrics")
plt.tight_layout()
plt.show()

# Building the ctmc and running it

In [ ]:
from simulation.unfold_events import rename_repeating_events

df, final_states = rename_repeating_events(df,epsilon,final_states)

In [ ]:
subset_el = pm4py.convert_to_event_log(df)
subset_el = log_parser.add_start_end(subset_el)
dfg, start_activities, end_activities = pm4py.discover_dfg(subset_el)
dfg["end", "start"] = 1

In [ ]:
pm4py.view_dfg(dfg, start_activities, end_activities)

In [ ]:
subset_el = pm4py.convert_to_dataframe(subset_el)

In [ ]:
data_transition_role_frequency = sim_util.get_transition_resource_dict(subset_el)

In [ ]:
data_mean_transition_role_time = {}
tuples_to_discard = set()
for k,v in data_transition_role_frequency.items():
    if k in ['start','end']:
        continue
    for k2,v2 in v.items():
        if k2 in ['start','end']:
            continue
        all_freq = 0
        for k3,v3 in v2.items():
            all_freq += v3
            if (k,k2,k3) in times_dictionary:
                times = times_dictionary[(k,k2,k3)]
                times = np.array(times)
                times = times/3600
                times = times[times != 0]
                if len(times) > 1: # only take times that have more than 1 value
                    expon_loc, expon_scale = scipy.stats.expon.fit(times)

                    # f = Fitter(times, distributions=['expon'])
                    # f.fit()
                    # best = f.get_best()['expon']
                    # expon_loc_fitter, expon_scale_fitter = best['loc'], best['scale']

                    if expon_scale>0: # do not take times that cannot be fit into an exponential
                        rate = 1/expon_scale
                        if k not in data_mean_transition_role_time:
                            data_mean_transition_role_time[k] = {}
                        if k2 not in data_mean_transition_role_time[k]:
                            data_mean_transition_role_time[k][k2] = {}
                        if k3 not in data_mean_transition_role_time[k][k2]:
                            data_mean_transition_role_time[k][k2][k3] = {
                                # 'loc': expon_loc_fitter,
                                # 'scale': expon_scale_fitter,
                                'loc': expon_loc,
                                'scale': expon_scale,
                                'lambda': rate
                            }
                    else:
                        print(f"[No exponential!] {k},{k2},{k3}")
                        tuples_to_discard.add((k,k2,k3))
                        print(times)
                else:
                    print(f"[No times!] {k},{k2},{k3}")
                    tuples_to_discard.add((k,k2,k3))
                    print(times)

In [ ]:
for (e_from,e_to,role) in tuples_to_discard:
    if e_from in data_transition_role_frequency:
        if e_to in data_transition_role_frequency[e_from]:
            if role in data_transition_role_frequency[e_from][e_to]:
                data_transition_role_frequency[e_from][e_to].pop(role)

In [ ]:
for e_from in data_transition_role_frequency.keys():
    for e_to in data_transition_role_frequency.keys():
        if (e_from == 'start' and e_to == 'start') or (e_from == 'end' and e_to == 'end'):
            data_transition_role_frequency[e_from].pop(e_to)

In [ ]:
def remove_empty_keys(d):
    """Recursively remove empty keys from a three-level nested dictionary."""
    if not isinstance(d, dict):
        return d  # Return non-dict values as they are

    cleaned_dict = {}
    for key, value in d.items():
        if isinstance(value, dict):
            cleaned_value = remove_empty_keys(value)  # Recursively clean sub-dictionaries
            if cleaned_value:  # Add only if not empty
                cleaned_dict[key] = cleaned_value
        elif value not in (None, "", [], {}, ()):  # Ignore empty values
            cleaned_dict[key] = value

    return cleaned_dict

data_transition_role_frequency = remove_empty_keys(data_transition_role_frequency)

In [ ]:
role_resources = sim_util.get_detailed_weighted_role(subset_el)

In [ ]:
role_trials = {k:int(v) for k,v in role_resources.items()}

In [ ]:
res = {}
out_frequency = {}
data_transition_role_prob = {}

for k,v in data_transition_role_frequency.items():
    if k in ['start','end']:
        continue
    out_freq = 0
    if k not in data_transition_role_prob:
        data_transition_role_prob[k] = {}

    for k2,v2 in v.items():
        if k2 in ['start','end']:
            continue
        all_freq = 0

        if k2 not in data_transition_role_prob[k]:
            data_transition_role_prob[k][k2] = {}

        if k not in res:
            res[k] = {}
        if k2 not in res[k]:
            for k3,v3 in v2.items():
                if k3 not in data_transition_role_prob[k][k2]:
                    data_transition_role_prob[k][k2][k3] = v3
                all_freq += v3
            res[k][k2] = all_freq
            out_freq += all_freq
        out_frequency[k] = out_freq

for k,v in res.items():
    for k2,v2 in v.items():
        res[k][k2] = res[k][k2]/out_frequency[k]

for k,v in data_transition_role_prob.items():
    for k2,v2 in v.items():
        for k3,v3 in v2.items():
            data_transition_role_prob[k][k2][k3] = v3/out_frequency[k]

In [ ]:
view_resource_markov_chain(data_transition_role_prob)

In [ ]:
view_non_resource_markov_chain(res)

In [ ]:
states = set(subset_el['concept:name'].unique()).difference(set(['start','end']))
n = len(states)
i = 0
correspondence = {s:i for s,i in zip(states,range(len(states)))}
#TODO: make sure none of the final states have state = 0 in the prism program
non_final_states = list(states.difference(set(final_states)))
for s in final_states:
    if correspondence[s] == 0:
        correspondence[s] = correspondence[non_final_states[0]]
        correspondence[non_final_states[0]] = 0
correspondence

In [ ]:
from simulation.ctmc import create_prism_program_from_log

probabilities = create_prism_program_from_log(
                            correspondence,
                            final_states,
                            data_mean_transition_role_time,
                            role_resources,
                            data_transition_role_frequency,
                            role_trials,
                            'ctmc-bpic13.sm')
# print(probabilities)

In [ ]:
prism_program = stormpy.parse_prism_program('ctmc-bpic13.sm',prism_compat=True,simplify=True)

In [ ]:
model = stormpy.build_model(prism_program)
# print("Number of states: {}".format(model.nr_states))
# print("Number of transitions: {}".format(model.nr_transitions))
# print("Labels: {}".format(model.labeling.get_labels()))


# formula_str = f'R=? [F {labels}]'
# formula_str = f'Rmin=? [C]'

def get_result(model, prism_program):
    labels = ""
    for fs in final_states:
        labels += f'"q_terminal_{fs}" |'
    labels = labels[:-2]
    formula_str = f'Tmin=? [F {labels}]'
    print(final_states)
    properties = stormpy.parse_properties(formula_str, prism_program)
    result = stormpy.model_checking(model, properties[0])
    initial_state = model.initial_states[0]
    result = result.at(initial_state)
    print(f"Hours: {result}")
    if result<np.inf:
        if time_unit == 'hours':
            print(f"Duration: {timedelta(hours=result)}")
        else:
            print(f"Duration: {timedelta(seconds=result)}")
    return result

result = get_result(model,prism_program)

# Analysis between ground truth, result and metrics

In [ ]:
metrics_all

In [ ]:
import random

durations = []
# x = list(range(1,50))
samples = 500
for i in range(samples):
    regression_role_trials = {}
    for k,v in role_trials.items():
        random_resource_number = abs(random.gauss(v,v/2))
        regression_role_trials[k] = random_resource_number
    probabilities = create_prism_program_from_log(
                            correspondence,
                            final_states,
                            data_mean_transition_role_time,
                            role_resources,
                            data_transition_role_frequency,
                            regression_role_trials,
                            'ctmc.sm')
    prism_program = stormpy.parse_prism_program('ctmc.sm', prism_compat=True, simplify=True)
    model = stormpy.build_model(prism_program)
    labels = ""
    for fs in final_states:
        labels += f'"q_terminal_{fs}" |'
    labels = labels[:-2]

    formula_str = f'Tmin=? [F {labels}]'
    properties = stormpy.parse_properties(formula_str, prism_program)
    result = stormpy.model_checking(model, properties[0])
    initial_state = model.initial_states[0]
    result = result.at(initial_state)
    durations.append({**regression_role_trials, "duration": result})
    # print(f'{i}/{samples}')

In [ ]:
durations_df = pd.DataFrame(durations)
durations_df

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

# Example: Load your DataFrame
# df = pd.read_csv("your_data.csv")

def run_linear_regression(df, target_column, fit_intercept=True):
    # Split into X (features) and y (target)
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Fit model
    model = LinearRegression(fit_intercept=fit_intercept)
    model.fit(X, y)

    # Print model coefficients
    intercept = model.intercept_
    coef = model.coef_

    print("Intercept:", intercept if fit_intercept else "Not used")
    print("Coefficients:")
    for col, weight in zip(X.columns, coef):
        print(f"{col}: {weight:.4f}")

    return model, X.columns, coef

def rank_features_by_importance(X, coef):
    importance_df = pd.DataFrame({
        'Feature': X,
        'Coefficient': coef,
        'Importance (abs)': abs(coef)
    })
    return importance_df.sort_values(by='Importance (abs)', ascending=False)

# Run regression
model, features, coefs = run_linear_regression(durations_df, 'duration', fit_intercept=False)

# Rank features
ranking = rank_features_by_importance(features, coefs)
print("\nFeature Ranking:\n", ranking)

# Other stuff and old stuff not used

In [ ]:
# Extract time deltas in desired unit
time_diffs = {}
for case_id, group in df.groupby("case:concept:name"):
    events = group["concept:name"].tolist()
    times = group["time:timestamp"].tolist()
    for i in range(len(events) - 1):
        pair = (events[i], events[i + 1])
        delta_time = (times[i + 1] - times[i]).total_seconds() * time_factor
        time_diffs.setdefault(pair, []).append(delta_time)

In [ ]:
# Plot histogram and exponential fits
fig, axes = plt.subplots(1, len(time_diffs), figsize=(14, 5))

for ax, (pair, deltas) in zip(axes, time_diffs.items()):
    deltas = np.array(deltas)
    mean_time = deltas.mean()
    n = len(deltas)

    # Manual exponential fit
    rate_manual = 1 / mean_time
    ci_half_width = t.ppf(0.975, df=n-1) * sem(deltas)
    lower_rate = 1 / (mean_time + ci_half_width)
    upper_rate = 1 / (mean_time - ci_half_width)

    # Scipy fit with loc fixed to 0
    loc, scale = expon.fit(deltas, floc=0)
    rate_scipy = 1 / scale

    # Plot histogram
    sns.histplot(deltas, bins=10, stat="density", ax=ax, color="skyblue", label="Empirical")

    # X values for plotting
    x_vals = np.linspace(0, max(deltas) * 1.2, 200)

    # PDF curves
    # ax.plot(x_vals, expon.pdf(x_vals, scale=1 / rate_manual), color="darkblue",
    #         label=f"Fitted λ (manual) = {rate_manual:.4f}")
    ax.plot(x_vals, expon.pdf(x_vals, scale=1 / rate_scipy), linestyle="-.", color="purple",
        label=f"Fitted λ (scipy) = {rate_scipy:.4f}")
    ax.fill_between(x_vals,
                    expon.pdf(x_vals, scale=1 / lower_rate),
                    expon.pdf(x_vals, scale=1 / upper_rate),
                    color="gray", alpha=0.3, label="95% CI band")

    # Labels and titles
    ax.set_title(f"{pair[0]} → {pair[1]}")
    ax.set_xlabel(f"Time ({time_unit})")
    ax.set_ylabel("Density")
    ax.legend()

plt.tight_layout()
plt.show()

In [ ]:
def extract_times_event_log(log_input):
    if type(log_input) == pd.DataFrame:
        log = pm4py.convert_to_event_log(deepcopy(log_input))
    else:
        log = deepcopy(log_input)
    times = []
    for trace in log:
        start = trace[0]['time:timestamp']
        end = trace[len(trace)-1] ['time:timestamp']
        time = end - start
        times.append(time.total_seconds())
    return times

def extract_times_with_future(log_input):
    times_dictionary = {}
    if type(log_input) == pd.DataFrame:
        log = pm4py.convert_to_event_log(deepcopy(log_input))
    else:
        log = deepcopy(log_input)
    for trace in log:
        first = True
        for next_event in trace:
            if not first and next_event['concept:name'] != 'end' and event['concept:name'] != 'start':
                time = next_event['time:timestamp'] - event['time:timestamp']
                if not (event['concept:name'], next_event['concept:name']) in times_dictionary.keys():
                    times_dictionary[(event['concept:name'], next_event['concept:name'])] = [time.total_seconds()]
                else:
                    times_dictionary[(event['concept:name'], next_event['concept:name'])].append(time.total_seconds())
            event = next_event
            first = False
    return times_dictionary

In [ ]:
if False:
    semi_markov_json = mc_apply(subset_el)
    view_markov_chain(semi_markov_json)

In [ ]:
# prism_program = stormpy.parse_prism_program('ctmc_simple_choice.sm',prism_compat=True,simplify=True)
prism_program = stormpy.parse_prism_program('ctmc_basic.sm',prism_compat=True,simplify=True)
model = stormpy.build_model(prism_program)
# print("Number of states: {}".format(model.nr_states))
# print("Number of transitions: {}".format(model.nr_transitions))
# print("Labels: {}".format(model.labeling.get_labels()))
labels = ""
for fs in final_states:
    labels += f'"q_terminal_{fs}" |'
labels = labels[:-2]

formula_str = f'R=? [F {labels}]'
properties = stormpy.parse_properties(formula_str, prism_program)
result = stormpy.model_checking(model, properties[0])
initial_state = model.initial_states[0]
result = result.at(initial_state)
print(f"Hours: {result}")
if result<np.inf:
    print(f"Duration: {timedelta(seconds=result)}")

In [ ]:
data_mean_transition_role_time

In [ ]:
pm4py.get_cycle_time(subset_el)

In [ ]:
pm4py.get_all_case_durations(subset_el)

In [ ]:
mean, median, margin_of_error = sim_util.get_pm4py_reference_times(subset_el)
print(timedelta(seconds=median))
print(timedelta(seconds=mean))
print(timedelta(seconds=margin_of_error))

In [ ]:
middle_mean = np.mean(times_dictionary[('Start','Middle','R1')]) + np.mean(times_dictionary[('Middle','End','R1')])
start_end_mean = np.mean(times_dictionary[('Start','End','R1')])
mean_all = np.mean([middle_mean, start_end_mean])
print(timedelta(seconds=mean_all))